In [1]:
import pandas as pd
import numpy as np

# Data Viz library
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# อ่านไฟล์ Fraud
Fraud= pd.read_excel('FraudTrain.xlsx')
Fraud.head()

,ID,CusID,Products,Quant,Value,Insp
0,207768,v955,p3725,48025.0,714335.0,ok
1,6434,v169,p1125,538.0,2965.0,ok
2,290584,v252,p994,113.0,1160.0,ok
3,210186,v4985,p3994,131.0,1190.0,ok
4,19882,v2752,p2272,105.0,1130.0,ok


In [3]:
# function check ค่าว่าง ถ้ามีค่าว่าง return true
Fraud.replace([np.inf, -np.inf], np.nan)
Fraud.isnull().any() 

ID          False
CusID       False
Products    False
Quant        True
Value        True
Insp        False
dtype: bool

In [4]:
# Fill in missing value and replace by mean value

Fraud['Quant'].mean()

def Quant  (x):
     if pd.isnull(x):
        return Fraud['Quant'].mean()
     else:
        return x
    
Fraud['Quant'] = Fraud['Quant'].apply(Quant)

Fraud.isnull().any()

ID          False
CusID       False
Products    False
Quant       False
Value        True
Insp        False
dtype: bool

In [5]:
# Fill in missing value and replace by mean value

Fraud['Value'].mean()

def Value (x):
     if pd.isnull(x):
        return Fraud['Value'].mean()
     else:
        return x
    
Fraud['Value'] = Fraud['Value'].apply( Value)

Fraud.isnull().any()

ID          False
CusID       False
Products    False
Quant       False
Value       False
Insp        False
dtype: bool

In [6]:
X = Fraud.drop('Insp',axis=1)
Y = Fraud['Insp']

In [7]:
X.head()

,ID,CusID,Products,Quant,Value
0,207768,v955,p3725,48025.0,714335.0
1,6434,v169,p1125,538.0,2965.0
2,290584,v252,p994,113.0,1160.0
3,210186,v4985,p3994,131.0,1190.0
4,19882,v2752,p2272,105.0,1130.0


In [8]:
Y.head()

0    ok
1    ok
2    ok
3    ok
4    ok
Name: Insp, dtype: object

In [9]:
X = pd.get_dummies(X, columns=['CusID', 'Products'])
X.head()

,ID,Quant,Value,CusID_v1,CusID_v10,CusID_v1002,CusID_v1003,CusID_v1004,CusID_v1005,CusID_v1008,...,Products_p983,Products_p985,Products_p989,Products_p99,Products_p991,Products_p993,Products_p994,Products_p996,Products_p997,Products_p998
0,207768,48025.0,714335.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6434,538.0,2965.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,290584,113.0,1160.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,210186,131.0,1190.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,19882,105.0,1130.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# นำเข้า train test split function

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=55)

In [11]:
# นำเข้าตัวแบบ Gradient Boosting

import pandas
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier


In [12]:
# Define ตัวแบบ ชื่อ GB

GB = GradientBoostingClassifier(n_estimators= 200, random_state= 101)

In [13]:
# Train Model

GB.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='auto',
                           random_state=101, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [14]:
# ทำนายค่ากับชุด X_test

pred = GB.predict(X_test)

In [15]:
# นำเข้า classification report และ confusion matrix

from sklearn.metrics import classification_report, confusion_matrix

In [16]:
# สั่งพิมพ์รายงาน ค่าการวัดผล และประสิทธิภาพการทำนายของตัวแบบ และค่าใน confusion matrix

print(classification_report(y_test, pred))
print('\n')
print(confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

       fraud       0.98      0.30      0.46       219
          ok       0.95      1.00      0.97      2754

    accuracy                           0.95      2973
   macro avg       0.97      0.65      0.71      2973
weighted avg       0.95      0.95      0.93      2973



[[  65  154]
 [   1 2753]]


In [17]:
def find_error_classification(y_test, pred):
    N = y_test.shape[0]
    error = (100/N)*(y_test!= pred).sum()
    return error

In [18]:
error_Test =find_error_classification(y_test, pred)

In [19]:
error_Test

5.213588967373024

In [20]:
# นำเข้า GridSearchCV
# Update คำสั่ง GridSearch  python 3

#from sklearn.model_selection import GridSearchCV

In [21]:
# กำหนด parameter ที่จะให้ GridSearch ทำการทดลอง

# Use scikit-learn to grid search
#n_estimators = [100, 500,1000]
#max_depth = [3, 4, 5]
#min_samples_split = [2,4,6]

In [22]:
# Create parameter options
#Gdict = dict(n_estimators = n_estimators, 
            # max_depth = max_depth, min_samples_split = min_samples_split)

In [23]:
# หากต้องการดู parameter ต่างๆ ของ GB ให้คลิ๊ก Shift+Tab หลังคำว่า "GridSearchCV"

#grid = GridSearchCV(estimator = GB, param_grid = Gdict)

In [24]:
# ทำการทดลองสร้างตัวแบบ Training set ด้วย grid set ที่กำหนดไว้

#grid.fit(X_train, y_train)

In [25]:
# Voting Ensemble for Classification
import pandas
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier


# create the sub models
model1 = DecisionTreeClassifier()
model2 = KNeighborsClassifier(n_neighbors = 1)
model3 = LogisticRegression()
model4 = RandomForestClassifier(max_features = 11, random_state = 55)

model = VotingClassifier(estimators=[('DT', model1), ('Knn', model2)
                                    , ('LG', model3),('RF',model4)], voting='hard')
model.fit(X_train,y_train)
model.score(X_test,y_test)

C:\Python Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Python Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9764547595021863

In [26]:
# ทำนายค่ากับชุด X_test

pred = model.predict(X_test)

In [27]:

# สั่งพิมพ์รายงาน ค่าการวัดผล และประสิทธิภาพการทำนายของตัวแบบ และค่าใน confusion matrix

print(classification_report(y_test, pred))
print('\n')
print(confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

       fraud       0.93      0.74      0.82       219
          ok       0.98      1.00      0.99      2754

    accuracy                           0.98      2973
   macro avg       0.95      0.87      0.90      2973
weighted avg       0.98      0.98      0.98      2973



[[ 161   58]
 [  12 2742]]


In [28]:
# แปลงค่า array ของผล pred ให้เป็น DataFrame
# ตั้งชื่อ columns ว่า 'Predict'

columns_new = ['Predict']

# pass in array and columns
prediction = pd.DataFrame(pred, columns=columns_new)
prediction.head()

,Predict
0,ok
1,ok
2,ok
3,ok
4,ok


In [29]:
prediction

,Predict
0,ok
1,ok
2,ok
3,ok
4,ok
5,ok
6,ok
7,ok
8,ok
9,ok


In [30]:
y_test

5607       ok
4340       ok
1321       ok
2027       ok
2008       ok
8953       ok
3385       ok
537        ok
3135       ok
3582       ok
7505       ok
6113       ok
419        ok
4643       ok
35         ok
6275       ok
1704       ok
4602       ok
2354       ok
3390       ok
2657       ok
1605       ok
4952       ok
9893    fraud
696        ok
543        ok
4917       ok
1721       ok
4604       ok
2571       ok
        ...  
6260       ok
7522       ok
5897       ok
6450       ok
2593       ok
4303       ok
9663    fraud
5636       ok
5926       ok
5900       ok
9081       ok
3187       ok
9416    fraud
1750       ok
2322       ok
9002       ok
506        ok
8394       ok
9164    fraud
1346       ok
8239       ok
4668       ok
5134       ok
5892       ok
8328       ok
4469       ok
5779       ok
3869       ok
4255       ok
3983       ok
Name: Insp, Length: 2973, dtype: object

In [31]:
y_test.reset_index(drop=True, inplace=True)

y_test

0          ok
1          ok
2          ok
3          ok
4          ok
5          ok
6          ok
7          ok
8          ok
9          ok
10         ok
11         ok
12         ok
13         ok
14         ok
15         ok
16         ok
17         ok
18         ok
19         ok
20         ok
21         ok
22         ok
23      fraud
24         ok
25         ok
26         ok
27         ok
28         ok
29         ok
        ...  
2943       ok
2944       ok
2945       ok
2946       ok
2947       ok
2948       ok
2949    fraud
2950       ok
2951       ok
2952       ok
2953       ok
2954       ok
2955    fraud
2956       ok
2957       ok
2958       ok
2959       ok
2960       ok
2961    fraud
2962       ok
2963       ok
2964       ok
2965       ok
2966       ok
2967       ok
2968       ok
2969       ok
2970       ok
2971       ok
2972       ok
Name: Insp, Length: 2973, dtype: object

In [32]:
result = pd.concat([y_test,prediction], axis = 1)
result


,Insp,Predict
0,ok,ok
1,ok,ok
2,ok,ok
3,ok,ok
4,ok,ok
5,ok,ok
6,ok,ok
7,ok,ok
8,ok,ok
9,ok,ok
